# Inspect DBT Models

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

View contents of Greenery DuckDB database that were created in my personal Snowflake schema using `dbt run` for week 1 project.

### Assumptions

1. Since the week 1 assignment was submitted late, the snapshots in my personal schema should be for weeks 1 and 2. <span style="color:red"><b>This assumption is not enforced here and the snapshot is assumed to be for week 1</b></span>.                                                                      
### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
schema_name = os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA")

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=schema_name,
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to database

In [ ]:
%sql engine --alias connection

## Explore Data

### Show All Tables in Database

In [ ]:
%%sql
SELECT table_catalog,
       table_schema,
       table_name,
       table_type,
       is_insertable_into,
       is_typed
FROM information_schema.tables
WHERE table_type LIKE '%TABLE'
AND table_schema = '{{schema_name.upper()}}'
ORDER BY table_catalog, table_name

### Show All Views in Database

In [ ]:
%%sql
SELECT table_catalog,
       table_schema,
       table_name,
       table_type,
       is_insertable_into,
       is_typed
FROM information_schema.tables
WHERE table_type LIKE '%VIEW'
AND table_schema = '{{schema_name.upper()}}'
ORDER BY table_catalog, table_name, table_schema

### Show Table and View Schema

### Staging

Staging view with addresses

In [ ]:
%%sql
DESCRIBE stg_postgres_addresses

Staging view with users

In [ ]:
%%sql
DESCRIBE stg_postgres_users

Staging view with promos

In [ ]:
%%sql
DESCRIBE stg_postgres_promos

Staging view with products

In [ ]:
%%sql
DESCRIBE stg_postgres_products

Staging view with orders

In [ ]:
%%sql
DESCRIBE stg_postgres_orders

Staging view with order items

In [ ]:
%%sql
DESCRIBE stg_postgres_order_items

Staging view with events

In [ ]:
%%sql
DESCRIBE stg_postgres_events

### Snapshots

Staging view with products

In [ ]:
%%sql
DESCRIBE stg_postgres_products

Snapshot table for monitoring changes to product inventory over time

In [ ]:
%%sql
DESCRIBE inventory_snapshot

### Run Queries

In [ ]:
%%sql
WITH t1 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_addresses' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_addresses
),
t2 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_users' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_users
),
t3 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_promos' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_promos
),
t4 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_products' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_products
),
t5 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_order_items' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_order_items
),
t6 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_orders' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_orders
),
t7 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'stg_postgres_events' AS materialization_name,
           'view' AS materialization_type,
           'model' AS dbt_element,
           'staging' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM stg_postgres_events
),
t8 AS (
    SELECT '{{schema_name}}' AS table_schema,
           'inventory_snapshot' AS materialization_name,
           'table' AS materialization_type,
           'model' AS dbt_element,
           'snapshot' AS dbt_element_type,
           COUNT(*) AS num_rows
    FROM inventory_snapshot
)
SELECT *
FROM (
    SELECT *
    FROM t1
    UNION
    SELECT *
    FROM t2
    UNION
    SELECT *
    FROM t3
    UNION
    SELECT *
    FROM t4
    UNION
    SELECT *
    FROM t5
    UNION
    SELECT *
    FROM t6
    UNION
    SELECT *
    FROM t7
    UNION
    SELECT *
    FROM t8
)
ORDER BY materialization_type ASC, dbt_element DESC

Show the snapshot table

In [ ]:
%%sql
SELECT *
FROM inventory_snapshot
LIMIT 4

Verify that no changes are visible in this table by counting the number of rows where `dbt_valid_to` does not contain a missing value

In [ ]:
%%sql
SELECT COUNT(*) AS num_changes
FROM inventory_snapshot
WHERE dbt_valid_to IS NOT NULL

## Disconnect

Close connection

In [ ]:
%sql --close connection